In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
import cross_fold
from midrc_dataset import midrc_challenge_dataset
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import os
from sklearn.metrics import cohen_kappa_score
from get_model import create_model
from config import config

%reload_ext autoreload
%autoreload 2

/u/spa-d2/grad/mah259/anaconda3/envs/research/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def train(epochs,model,device, train_loader, val_loader, criterion, optimizer, fold_number):
    best_vloss = 1_000_000.
    for e in range(epochs):
        print('EPOCH {}:'.format(e + 1))
        running_loss = 0.
        last_loss = 0.
        running_train_kappa=0.0
        avg_train_kappa=0.0
        model.train(True)

        for i, data in enumerate(train_loader):
            # Every data instance is an input + label pair
            inputs, labels = data

            # Zero your gradients for every batch!
            optimizer.zero_grad()

            # Make predictions for this batch
            outputs = model(inputs.to(device))
            labels=labels/24.0

            # Compute the loss and its gradients
            loss = criterion(outputs, labels.float().to(device).unsqueeze(1))
            loss.backward()

            # Adjust learning weights
            optimizer.step()
            outputs=outputs*24.0
            labels=labels*24.0
            
            outputs=torch.round(outputs)
            outputs=outputs.data.cpu().numpy()
            
            labels=labels.data.cpu().numpy()
            outputs=outputs.flatten()
            labels=labels.flatten()
            kappa_score_train=cohen_kappa_score(labels,outputs,weights='quadratic')
            running_train_kappa+=kappa_score_train
            # Gather data and report
            running_loss += loss.item()
            # if i % 10 == 9:
            #     last_loss = running_loss / 10 # loss per batch
            #     print('  batch {} loss: {}'.format(i + 1, last_loss))
            #     tb_x = e * len(train_loader) + i + 1
            #     #tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            #     running_loss = 0.


        
        avg_loss = running_loss/(i+1)
        avg_train_kappa=running_train_kappa/(i+1)

        
        running_vloss = 0.0
        running_kappa = 0.0
        model.eval()
        with torch.no_grad():
            for i, vdata in enumerate(val_loader):
                vinputs, vlabels = vdata
                voutputs = model(vinputs.to(device))
                vlabels=vlabels/24.0
                vloss = criterion(voutputs, vlabels.float().to(device).unsqueeze(1))
                voutputs=voutputs*24.0
                vlabels=vlabels*24.0
                voutputs=torch.round(voutputs)
                voutputs=voutputs.data.cpu().numpy()
                vlabels=vlabels.data.cpu().numpy()
                voutputs=voutputs.flatten()
                vlabels=vlabels.flatten()
                kappa_score=cohen_kappa_score(vlabels,voutputs,weights='quadratic')
                running_kappa+=kappa_score
                running_vloss += vloss
                

        avg_vloss = running_vloss / (i + 1)
        avg_val_kappa= running_kappa / (i + 1)
        
        print('LOSS train {} valid {}, Kappa train {} valid {}'.format(avg_loss, avg_vloss,avg_train_kappa,avg_val_kappa))

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = os.path.join('models','modelsave_fold_{}'.format(fold_number))
            torch.save(model.state_dict(), model_path)



In [3]:
def test(model,device,dataloader):
    with torch.no_grad():
        squared_error = 0
        absolute_error = 0

        for data in dataloader:
            inputs, labels = data

            outputs = model(inputs.to(device))

            squared_error += (outputs*torch.tensor(24.0).to(device) - labels.float().to(device).unsqueeze(1))**2
            absolute_error += torch.abs(outputs*torch.tensor(24.0).to(device)-labels.float().to(device).unsqueeze(1))
            
        #returns mae and rmse
        return absolute_error/len(dataloader), torch.sqrt(squared_error/len(dataloader))

In [4]:
# Training Loop

folds = cross_fold.create_folded_datasets("../data/resized_224X224/label_info/labels.json")

root_dir = '../data/resized_224X224'

annotations_file = 'MIDRC mRALE Mastermind Training Annotations_2079_20230428.csv'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 16
epochs = config['epochs']


if config['augment']:
    # Define the transformations
    transform = transforms.Compose([
        transforms.RandomRotation(10),                           # Randomly rotate the image within -10 to +10 degrees
        transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),# Randomly crop and resize the image to 224x224 pixels
        transforms.RandomHorizontalFlip(),                       # Randomly flip the image horizontally
        transforms.RandomApply([transforms.Lambda(lambda img: 
                                                  F.adjust_brightness(img, brightness_factor=torch.rand(1).item() + 0.5))], 
                                                  p=0.5),  # Random brightness adjustment
        transforms.ToTensor(),                                   # Convert the image to a tensor
        # TODO:Normalization?
    ])
else:
    transform = transforms.Compose([
        transforms.ToTensor(),
        # TODO:Normalization?
    ])

val_transform = transforms.Compose([transforms.ToTensor()])


for f_i,fold in enumerate(folds):
    print("FOLD: ",f_i+1)
    train_list, val_list = fold

    model = create_model(config=config)
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters())

    criterion = torch.nn.MSELoss()

    train_dataset = midrc_challenge_dataset(root_dir, annotations_file, transform, fp_list = train_list)
    val_dataset = midrc_challenge_dataset(root_dir, annotations_file, val_transform, fp_list = val_list)
    
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=True)

    #Training per fold
    train(epochs,model,device,train_loader,val_loader, criterion, optimizer, f_i)    

Fold:  1  Train Indices:  2078  Val indices:  520
Fold:  2  Train Indices:  2096  Val indices:  502
Fold:  3  Train Indices:  2050  Val indices:  548
Fold:  4  Train Indices:  2072  Val indices:  526
Fold:  5  Train Indices:  2096  Val indices:  502


/u/spa-d2/grad/mah259/anaconda3/envs/research/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/u/spa-d2/grad/mah259/anaconda3/envs/research/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EPOCH 1:
LOSS train 0.13785449859614557 valid 0.19194571673870087, Kappa train 0.05088007636145116 valid 0.2262654597842928
EPOCH 2:
LOSS train 0.10868135321025665 valid 0.11101017147302628, Kappa train 0.23276807193157145 valid 0.3584817337876917
EPOCH 3:
LOSS train 0.09946088733581396 valid 0.10054195672273636, Kappa train 0.30811323923124306 valid 0.5713177378350045
EPOCH 4:
LOSS train 0.08703128503492245 valid 0.16995783150196075, Kappa train 0.4428359642208689 valid 0.18090501423955607
EPOCH 5:
LOSS train 0.0732894787994715 valid 0.08643194288015366, Kappa train 0.5457211607134518 valid 0.6296977552879096
EPOCH 6:
LOSS train 0.07492348213608448 valid 0.07981036603450775, Kappa train 0.5403113404015518 valid 0.6022760100715193
EPOCH 7:
LOSS train 0.0700666304391164 valid 0.06076836213469505, Kappa train 0.5776868066204335 valid 0.7028635160028557
EPOCH 8:
LOSS train 0.06624685563147067 valid 0.06850048154592514, Kappa train 0.5988265552265201 valid 0.643672926691326
EPOCH 9:
LOSS t

/u/spa-d2/grad/mah259/anaconda3/envs/research/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/u/spa-d2/grad/mah259/anaconda3/envs/research/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EPOCH 1:
LOSS train 0.1327347114454699 valid 0.0998387336730957, Kappa train 0.23730905032110844 valid 0.25414347589391056
EPOCH 2:
LOSS train 0.11118936757663735 valid 0.08794699609279633, Kappa train 0.31514419386892345 valid 0.30604073561500156
EPOCH 3:
LOSS train 0.09659040395085139 valid 0.08613785356283188, Kappa train 0.45000088695135204 valid 0.34977610313280416
EPOCH 4:
LOSS train 0.08062073895494447 valid 0.06834349781274796, Kappa train 0.5549364350915202 valid 0.4858225900425834
EPOCH 5:
LOSS train 0.07191210838731464 valid 0.0590057298541069, Kappa train 0.6115503080170187 valid 0.5512690328978143
EPOCH 6:
LOSS train 0.06452819740556123 valid 0.06601826101541519, Kappa train 0.6581815709818674 valid 0.5924885038302788
EPOCH 7:
LOSS train 0.06572264226736459 valid 0.06739597767591476, Kappa train 0.6574133293109627 valid 0.580669697205147
EPOCH 8:
LOSS train 0.06252951524753607 valid 0.060270700603723526, Kappa train 0.6687709811311452 valid 0.6245200724204927
EPOCH 9:
LOSS

/u/spa-d2/grad/mah259/anaconda3/envs/research/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/u/spa-d2/grad/mah259/anaconda3/envs/research/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EPOCH 1:
LOSS train 0.13026717934728593 valid 0.12379279732704163, Kappa train 0.17518904472408814 valid 0.3613283864388003
EPOCH 2:
LOSS train 0.11075745187988577 valid 0.13864833116531372, Kappa train 0.267833628170497 valid 0.2414199204470281
EPOCH 3:
LOSS train 0.09464935582968616 valid 0.19056332111358643, Kappa train 0.3865706666765274 valid 0.17929697894263766
EPOCH 4:
LOSS train 0.09306003491199294 valid 0.10279406607151031, Kappa train 0.4097924810009442 valid 0.3506767227352909
EPOCH 5:
LOSS train 0.07979222937205503 valid 0.09775731712579727, Kappa train 0.5100257230622234 valid 0.4156204749583088
EPOCH 6:
LOSS train 0.07285419928790755 valid 0.10163211077451706, Kappa train 0.55008896812897 valid 0.5628404244877669
EPOCH 7:
LOSS train 0.07049975789679114 valid 0.05844782292842865, Kappa train 0.5750111376047566 valid 0.655117729575568
EPOCH 8:
LOSS train 0.06707545121510823 valid 0.0697299912571907, Kappa train 0.6069220993493246 valid 0.6423134903980076
EPOCH 9:
LOSS train

/u/spa-d2/grad/mah259/anaconda3/envs/research/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/u/spa-d2/grad/mah259/anaconda3/envs/research/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EPOCH 1:


In [ ]:
#Testing Loop
for f_i,fold in enumerate(folds):

    train_list, val_list = fold

    val_dataset = midrc_challenge_dataset(root_dir, annotations_file, val_transform, fp_list = val_list)
    val_loader = DataLoader(val_dataset, batch_size = 1, shuffle=True)

    model = create_model()
    model_pth = os.path.join('models','modelsave_fold_{}'.format(f_i))
    model.load_state_dict(torch.load(model_pth))
    model.to(device)
    model.eval()

    avg_mae, avg_rmse = test(model,device,val_loader)

    print("Fold: ",f_i+1, " MAE:", avg_mae.item() , " RMSE: ", avg_rmse.item())

Fold:  1  MAE: 5.070911407470703  RMSE:  6.121790885925293
Fold:  2  MAE: 5.229495525360107  RMSE:  6.513065814971924
Fold:  3  MAE: 4.875302791595459  RMSE:  6.091127872467041
Fold:  4  MAE: 5.047328472137451  RMSE:  6.208610534667969
Fold:  5  MAE: 4.741214752197266  RMSE:  6.251406192779541
